In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [5]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
import os
from langchain_openai import ChatOpenAI  # Notez le changement ici

# Définition de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0xUZ6aBpi1 2mjq9C0Iyh8mq8WeoK0yCT3BlbkFJ6nuNH0hRFwAy9HhFfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"

# Création du modèle
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Notez que c'est 'model' et non 'model_name' dans les versions récentes
    temperature=0
)

# Memory-of-Thought Prompting

Memory-of-Thought Prompting, introduced by Li and Qiu (2023), is a powerful approach that enhances language models (LLMs) by dynamically retrieving and leveraging similar examples from a pre-built repository.

## How Does Memory-of-Thought Prompting Work?

Imagine you're a mathematician who keeps a notebook of solved problems. When faced with a new challenge, instead of starting from scratch, you first look through your notebook for similar problems you've already solved.

Memory-of-Thought Prompting follows this intuitive strategy in three key steps:

1. **Knowledge Repository Creation**: Before deployment, the system builds a collection of unlabeled training examples processed with Chain-of-Thought reasoning, creating a "memory bank" of solved problems.

2. **Similarity-Based Retrieval**: When presented with a new problem, the system retrieves the most semantically similar examples from its memory bank, identifying those with relevant reasoning patterns.

3. **Reasoning Transfer**: The model adapts the reasoning patterns from the retrieved examples to solve the new problem, effectively transferring knowledge across similar problem structures.

This technique has demonstrated substantial improvements on benchmarks involving arithmetic reasoning, commonsense reasoning, and factual reasoning tasks by leveraging the power of analogical problem-solving.

In [8]:
memory_of_thought_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Memory-of-Thought Prompting examples based on user questions.
    
    Memory-of-Thought Prompting is a specialized technique within Few-Shot Chain-of-Thought (CoT) prompting that includes the following steps:
    1. Creates a repository of unlabeled training exemplars with CoT reasoning
    2. Processes these exemplars before test time using CoT inference
    3. At test time, retrieves the most similar instances to the test sample
    4. Incorporates the retrieved instances as context for solving the test problem
    5. Generates a solution leveraging the "memory" of similar problems
    
    This approach enhances LLM performance by dynamically selecting relevant examples from a memory bank of solved problems.
    
    Please structure your response in the following format:
    
    Step 1: Memory Bank Creation
    [Provide 3-5 unlabeled exemplars with questions and CoT reasoning]
    
    Step 2: Similarity Analysis
    [Identify which exemplars from the memory bank are most similar to the test question]
    
    Step 3: Context Retrieval
    [Extract the relevant reasoning patterns from the similar exemplars]
    
    Step 4: Memory-Augmented Reasoning
    [Apply the retrieved reasoning patterns to the test question]
    
    Step 5: Final Solution
    [Comprehensive answer to the original question using insights from the memory-augmented reasoning]
    
    Example:
    Question: "If a rectangle has a length of 12 meters and a width of 8 meters, what is the area of the rectangle and its perimeter?"
    
    Memory Bank Creation:
    Exemplar 1: 
    Q: What is the area of a square with sides of length 5 cm?
    A: To find the area of a square, I need to multiply the length of one side by itself.
    Area = side × side = side²
    Area = 5 cm × 5 cm = 25 cm²
    Therefore, the area of the square is 25 square centimeters.
    
    Exemplar 2:
    Q: Find the perimeter of a rectangle with length 9 inches and width 4 inches.
    A: To find the perimeter of a rectangle, I need to add all four sides.
    Perimeter = 2 × length + 2 × width
    Perimeter = 2 × 9 inches + 2 × 4 inches
    Perimeter = 18 inches + 8 inches = 26 inches
    Therefore, the perimeter of the rectangle is 26 inches.
    
    Exemplar 3:
    Q: What is the area of a triangle with base 6 meters and height 8 meters?
    A: To find the area of a triangle, I use the formula Area = (1/2) × base × height.
    Area = (1/2) × 6 meters × 8 meters
    Area = (1/2) × 48 square meters
    Area = 24 square meters
    Therefore, the area of the triangle is 24 square meters.
    
    Similarity Analysis:
    For our question about a rectangle's area and perimeter, Exemplar 1 involves area calculation but for a square, and Exemplar 2 involves perimeter calculation for a rectangle. Both are relevant. Exemplar 3 is less relevant as it deals with a triangle.
    
    Context Retrieval:
    From Exemplar 1: Area calculation requires multiplying length by width for a rectangle.
    From Exemplar 2: Perimeter calculation uses the formula 2 × length + 2 × width.
    
    Memory-Augmented Reasoning:
    For our rectangle with length 12 meters and width 8 meters:
    
    To calculate the area, I'll adapt the reasoning from Exemplar 1:
    Area = length × width
    Area = 12 meters × 8 meters = 96 square meters
    
    To calculate the perimeter, I'll use the formula from Exemplar 2:
    Perimeter = 2 × length + 2 × width
    Perimeter = 2 × 12 meters + 2 × 8 meters
    Perimeter = 24 meters + 16 meters = 40 meters
    
    Final Solution:
    For a rectangle with length 12 meters and width 8 meters:
    
    Area calculation:
    Area = length × width
    Area = 12 m × 8 m = 96 m²
    
    Perimeter calculation:
    Perimeter = 2 × length + 2 × width
    Perimeter = 2 × 12 m + 2 × 8 m
    Perimeter = 24 m + 16 m = 40 m
    
    Therefore, the area of the rectangle is 96 square meters, and its perimeter is 40 meters.
    """),
    ("human", "{question}"),
    ("assistant", """
    I'll solve this problem using Memory-of-Thought Prompting to leverage similar examples:
    
    Step 1: Memory Bank Creation
    [Provide 3-5 relevant exemplars with detailed reasoning paths that relate to the problem domain]
    
    Step 2: Similarity Analysis
    [Identify which exemplars from the memory bank are most conceptually similar to the current problem]
    
    Step 3: Context Retrieval
    [Extract the key reasoning patterns and formulas from the similar exemplars]
    
    Step 4: Memory-Augmented Reasoning
    [Apply the retrieved patterns to the current problem with clear step-by-step reasoning]
    
    Step 5: Final Solution
    [Comprehensive solution that combines the retrieved knowledge with problem-specific reasoning]
    """)
])

 

For Memory-of-Thought Prompting, the most suitable questions are those where retrieving and applying reasoning patterns from similar examples can significantly enhance model capabilities. Here are types of questions that would be particularly well-suited to this approach:

### Pattern Recognition Problems
- Mathematical problems that follow established formulas or methods
- Algorithmic challenges with recognizable solution patterns
- Tasks requiring application of specific procedures or templates
- Questions with identifiable reasoning structures that can be reused

### Procedural or Multi-Step Problems
- Complex calculations with step-by-step processes
- Financial problems using standard formulas
- Scientific procedures with defined methodologies
- Any problem requiring a sequence of well-defined operations

### Domain-Specific Knowledge Application
- Physics problems requiring application of specific laws
- Chemistry problems involving standard reactions or equations
- Computer science problems using established algorithms
- Financial calculations using standard accounting principles

### Problems with Generalizable Solutions
- Questions where past solutions can directly inform new problems
- Scenarios where established frameworks can be adapted
- Problems that benefit from analogical reasoning from similar cases
- Questions requiring transfer learning from one domain to a related one

### Concrete Examples of Questions

1. "If a rectangle has a length of 15 meters and a width of 8 meters, what is its area and perimeter?"
   - This question follows a standard pattern that can be easily retrieved from memory

2. "If I invest $10,000 at an annual interest rate of 5% compounded monthly, how much will my investment be worth after 10 years?"
   - Financial calculation that benefits from reusing similar compound interest calculations

3. "What is the probability of drawing at least 2 red cards in 5 draws from a standard deck of 52 cards, without replacement?"
   - Statistical problem that follows established probability calculation patterns

4. "A projectile is launched at an angle of 45 degrees with an initial velocity of 20 m/s. How far will it travel horizontally before hitting the ground?"
   - Physics problem that can reuse kinematics equation patterns from similar problems

5. "Using the quadratic formula, solve for x in the equation 3x² - 5x - 2 = 0."
   - Algebraic problem where memorized procedure application is beneficial

Memory-of-Thought Prompting is particularly effective for these types of questions because:

- It builds a "knowledge bank" of solved examples that can be retrieved when needed
- It leverages similarity between current problems and previously encountered ones
- It reduces computational effort by adapting existing solutions rather than solving from scratch
- It mimics human expert behavior of recognizing patterns and applying known solutions
- It excels at problems with clear, structured approaches that can be reused across similar instances

This approach is especially valuable in educational contexts, standardized testing, scientific domains, and any area where recognizing and applying patterns from similar examples leads to more efficient problem-solving. The key advantage is its ability to dynamically retrieve relevant knowledge and reasoning patterns rather than starting from first principles each time.

In [ ]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the Memory-of-Thought prompt and extract the final answer (Step 5)
def generate_memory_of_thought_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = memory_of_thought_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from Step 5
    final_answer_pattern = r'Step 5:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 5: Final Solution([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('memory_of_thought_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (STEP 5) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response,
        "memory_bank": extract_memory_bank(full_response)  # Extract memory bank for future reference
    }

# Helper function to extract the memory bank for potential reuse
def extract_memory_bank(response):
    memory_bank_pattern = r'Step 1: Memory Bank Creation([\s\S]*?)Step 2:'
    memory_bank_match = re.search(memory_bank_pattern, response)
    
    if memory_bank_match:
        return memory_bank_match.group(1).strip()
    else:
        return "Memory bank not found in the expected format."

# Definition of state with additional field for memory bank
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]
    memory_bank: Optional[str]  # Store memory bank for potential reuse

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_memory_of_thought", generate_memory_of_thought_node)

# Graph configuration
graph_builder.set_entry_point("generate_memory_of_thought")
graph_builder.add_edge("generate_memory_of_thought", END)
graph = graph_builder.compile()



In [10]:
# Usage example
inputs = {"messages": [HumanMessage(content="Marie gave half of her apples to Jean. Jean ate three apples," 
                                         " then gave half of what he had left to Pierre. " 
                                         "Pierre now has 4 apples. How many apples did Marie have at the beginning?")],
          "result_df": None,  # Initialize result_df to None
          "memory_bank": None}  # Initialize memory_bank to None


In [11]:
# Graph execution - only display Step 5
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_memory_of_thought":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'memory_of_thought_result.txt')")

Final Solution**  
Marie initially had 22 apples.  
To summarize:  
- Marie gave half (11 apples) to Jean.  
- Jean ate 3 apples (leaving him with 8 apples).  
- Jean then gave half of his remaining apples (4 apples) to Pierre, who now has 4 apples.  
Thus, the calculations confirm that Marie started with 22 apples.

(Complete results saved in 'memory_of_thought_result.txt')
